# Halite 

## 

In [1]:
import gym
import time
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.insert(0, "../Environment/")
import halite_env as Env


In [2]:
from importlib import reload
reload(Env)


<module 'halite_env' from '../Environment/halite_env.py'>

In [3]:
num_players = 1
map_size = 7 # 7 x 7 map

In [4]:
HEnv = Env.HaliteEnv(num_players, map_size)

In [5]:
def Single_ship_env_initialization(map_size, num_players = 1):
    HEnv = Env.HaliteEnv(num_players, map_size)
    return Env.SingleShipEnv(HEnv, 1, map_size)

def check_position(env):
    print(env.state[:,:,:])


In [6]:
env = Single_ship_env_initialization(map_size)
print(np.swapaxes(env.reset(), 0, 2))
check_position(env)
env.render()
#env.render().shape

[[[755 612  80 492 590 641  79]
  [150 833 889 614 228 915   7]
  [ 26 397 518  69 689 134  80]
  [295 609 618   0 259  55 915]
  [387 354 453 483 513 795   2]
  [166 955 200 218 457 299 981]
  [450 391 536 253 361 743 942]]

 [[  0   0   0   0   0   0   0]
  [  0   0   0   0   0   0   0]
  [  0   0   0   0   0   0   0]
  [  0   0   0   1   0   0   0]
  [  0   0   0   0   0   0   0]
  [  0   0   0   0   0   0   0]
  [  0   0   0   0   0   0   0]]

 [[  0   0   0   0   0   0   0]
  [  0   0   0   0   0   0   0]
  [  0   0   0   0   0   0   0]
  [  0   0   0   0   0   0   0]
  [  0   0   0   0   0   0   0]
  [  0   0   0   0   0   0   0]
  [  0   0   0   0   0   0   0]]

 [[  0   0   0   0   0   0   0]
  [  0   0   0   0   0   0   0]
  [  0   0   0   0   0   0   0]
  [  0   0   0   1   0   0   0]
  [  0   0   0   0   0   0   0]
  [  0   0   0   0   0   0   0]
  [  0   0   0   0   0   0   0]]

 [[  0   0   0   0   0   0   0]
  [  0   0   0   0   0   0   0]
  [  0   0   0   0   0   0   0]


In [7]:
env.step(1)
check_position(env)
state = env.state

[[[755   0   0   0   0]
  [150   0   0   0   0]
  [ 26   0   0   0   0]
  [295   0   0   0   0]
  [387   0   0   0   0]
  [166   0   0   0   0]
  [450   0   0   0   0]]

 [[612   0   0   0   0]
  [833   0   0   0   0]
  [397   0   0   0   0]
  [609   0   0   0   0]
  [354   0   0   0   0]
  [955   0   0   0   0]
  [391   0   0   0   0]]

 [[ 80   0   0   0   0]
  [889   0   0   0   0]
  [518   0   0   0   0]
  [618   0   0   0   0]
  [453   0   0   0   0]
  [200   0   0   0   0]
  [536   0   0   0   0]]

 [[492   0   0   0   0]
  [614   0   0   0   0]
  [ 69   0   0   0   0]
  [  0   0   0   1   0]
  [483   0   0   0   0]
  [218   0   0   0   0]
  [253   0   0   0   0]]

 [[590   0   0   0   0]
  [228   0   0   0   0]
  [689   0   0   0   0]
  [259   1   0   0   1]
  [513   0   0   0   0]
  [457   0   0   0   0]
  [361   0   0   0   0]]

 [[641   0   0   0   0]
  [915   0   0   0   0]
  [134   0   0   0   0]
  [ 55   0   0   0   0]
  [795   0   0   0   0]
  [299   0   0   0   0]
  [743

In [8]:
#np.nonzero(state[:,:,1])
#import emoji
#state[:,:,0]
#print(halite[1])
import emoji

In [9]:
halite = ([i for i in range(map_size) for j in range(map_size)],[j for i in range(map_size) for j in range(map_size)])
ships = np.nonzero(state[:,:,1])
cargo = np.nonzero(state[:,:,2])
factories = np.nonzero(state[:,:,3])

for _ in range(map_size):
    print('+-----', end='')
print('+', end='\n')
for i in range(map_size):

    for j in range(map_size):
        H = state[halite[0][i*map_size+j],halite[1][i*map_size+j],0]
        if H < 10:
            print('|   \x1b[2;35;46m{}\x1b[0m '.format(H), end='')
        elif H < 100:
            print('|  \x1b[2;35;46m{}\x1b[0m '.format(H), end='')
        else:
            print('| \x1b[2;35;46m{}\x1b[0m '.format(H), end='')
        
    print('|', end='\n')
    for j in range(map_size):
        print('|  ', end='')
        A = i in factories[1] and j in factories[0]
        B = i in ships[1] and j in ships[0]
        if A and B:
            print('🏰⛵️', end='')
        elif A:
            print('🏰 ', end='')
        elif B:
            print('⛵️ ', end='')
        else:
            print('   ', end='')
        
    print('|', end ='\n')
    for j in range(map_size):
        A = i in ships[1] and j in ships[0]
        B = i in cargo[1] and j in cargo[0]
        if A and B:
            C = state[cargo[0][i*map_size+j],cargo[1][i*map_size+j],2]
            if C < 10:
                print('|   \x1b[1;30;41m{}\x1b[0m '.format(C), end='')
            elif C < 100:
                print('|  \x1b[1;30;41m{}\x1b[0m '.format(C), end='')
            else:
                print('| \x1b[1;30;41m{}\x1b[0m '.format(C), end='')
        elif A:
            print('|  \x1b[1;30;41m0\x1b[0m  ', end='')
        else:
            print('|     ', end='')

    print('|', end='\n')
    
    for _ in range(map_size):
        print('+-----', end='')
    print('+', end='\n')

        
print('\033[93m'+"ooook"+'\033[0m')


+-----+-----+-----+-----+-----+-----+-----+
| 755 | 150 |  26 | 295 | 387 | 166 | 450 |
|     |     |     |     |     |     |     |
|     |     |     |     |     |     |     |
+-----+-----+-----+-----+-----+-----+-----+
| 612 | 833 | 397 | 609 | 354 | 955 | 391 |
|     |     |     |     |     |     |     |
|     |     |     |     |     |     |     |
+-----+-----+-----+-----+-----+-----+-----+
|  80 | 889 | 518 | 618 | 453 | 200 | 536 |
|     |     |     |     |     |     |     |
|     |     |     |     |     |     |     |
+-----+-----+-----+-----+-----+-----+-----+
| 492 | 614 |  69 |   0 | 483 | 218 | 253 |
|     |     |     |  🏰 |  ⛵️ |     |     |
|     |     |     |     |  0  |     |     |
+-----+-----+-----+-----+-----+-----+-----+
| 590 | 228 | 689 | 259 | 513 | 457 | 361 |
|     |     |     |     |     |     |     |
|     |     |     |     |     |     |     |
+-----+-----+-----+-----+-----+-----+-----+
| 641 | 915 | 134 |  55 | 795 | 299 | 743 |
|     |     |     |     |     |  

## Baselines

In [10]:
from stable_baselines.common.policies import MlpPolicy
from stable_baselines.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines.common import set_global_seeds
from stable_baselines import PPO2, ACKTR
import stable_baselines
import tensorflow as tf
reload(stable_baselines)
#from stable_baselines.logger import configure
#configure("/home/mango/Documents/haliteRL/tensorboard_ppo2", ['stdout','log','csv','tensorboard'])


MULTI = False
COMPARISON = False
obs_type = "raw"
n_timesteps = int(1e7)
map_size = 7

In [11]:
def make_env(map_size, rank, seed=1234):
    """
    Utility function for multiprocessed env.

    :param env_id: (str) the environment ID
    :param rank: (int) index of the subprocess
    :param seed: (int) the inital seed for RNG
    """

    def _init():
        env = Single_ship_env_initialization(map_size)
        env.seed(seed + rank)
        return env

    set_global_seeds(seed)
    return _init

def evaluate(model, num_steps=1000):
    """
    Evaluate a RL agent
    :param model: (BaseRLModel object) the RL Agent
    :param num_steps: (int) number of timesteps to evaluate it
    :return: (float) Mean reward
    """
    episode_rewards = [[0.0] for _ in range(env.num_envs)]
    obs = env.reset()
    for i in range(num_steps):
        # _states are only useful when using LSTM policies
        actions, _states = model.predict(obs)
        # here, action, rewards and dones are arrays
        # because we are using vectorized env
        obs, rewards, dones, info = env.step(actions)

        # Stats
        for j in range(env.num_envs):
            episode_rewards[j][-1] += rewards[j]
            if dones[j]:
                episode_rewards[j].append(0.0)

    mean_rewards = [0.0 for _ in range(env.num_envs)]
    n_episodes = 0
    for i in range(env.num_envs):
        mean_rewards[i] = np.mean(episode_rewards[i])
        n_episodes += len(episode_rewards[i])

    # Compute mean reward
    mean_reward = round(np.mean(mean_rewards), 1)
    print("Mean reward:", mean_reward, "Num episodes:", n_episodes)

    return mean_reward


def callback(locs, globs):
    hlt = locs['self'].env.env_method('pl_halite')
    print(hlt)
    tf.summary.scalar('Player_halite', hlt[0])
    merged = tf.summary.merge_all()
    train_writer = tf.summary.FileWriter('./tensorboard_ppo2/', sess.graph)

    return True


if MULTI:
    num_cpu = 32  # Number of processes to use
    # Create the vectorized environment
    env = SubprocVecEnv([make_env(map_size, i) for i in range(num_cpu)])
else:
    # env = SingleSnek(obs_type="rgb", n_food=3)
    env = Single_ship_env_initialization(map_size)
    # The algorithms require a vectorized environment to run
    env = DummyVecEnv([lambda: env])


ob = env.reset()



In [12]:
model = stable_baselines.PPO2(MlpPolicy, env, verbose=1, tensorboard_log="./tensorboard_ppo2/")


Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


In [13]:
mean_reward_before_train = evaluate(model, num_steps=1000)

Mean reward: -34.2 Num episodes: 3


In [14]:
if MULTI:
    # Multiprocessed RL Training
    start_time = time.time()
    model.learn(n_timesteps)
    total_time_multi = time.time() - start_time

    print(
        "Took {:.2f}s for multiprocessed version - {:.2f} FPS".format(
            total_time_multi, n_timesteps / total_time_multi
        )
    )
    
    if COMPARISON:
        # Single Process RL Training
        env = Single_ship_env_initialization(map_size)
        env = DummyVecEnv([lambda: env])

        single_process_model = PPO2(MlpPolicy, env, verbose=0)

        start_time = time.time()
        single_process_model.learn(n_timesteps)
        total_time_single = time.time() - start_time

        print(
            "Took {:.2f}s for single process version - {:.2f} FPS".format(
                total_time_single, n_timesteps / total_time_single
            )
        )

        print(
            "Multiprocessed training is {:.2f}x faster!".format(
                total_time_single / total_time_multi
            )
        )

    mean_reward = evaluate(model, num_steps=10000)
else:
    start_time = time.time()
    model.learn(n_timesteps, callback=callback)
    total_time_single = time.time() - start_time

    print(
        "Took {:.2f}s for training - {:.2f} FPS".format(
            total_time_single, n_timesteps / total_time_single
        )
    )

    mean_reward = evaluate(model, num_steps=10000)


--------------------------------------
| approxkl           | 0.00013025603 |
| clipfrac           | 0.0           |
| explained_variance | -0.231        |
| fps                | 290           |
| n_updates          | 1             |
| policy_entropy     | 1.386166      |
| policy_loss        | -0.0012364477 |
| serial_timesteps   | 128           |
| time_elapsed       | 2.38e-06      |
| total_timesteps    | 128           |
| value_loss         | 3.6016152     |
--------------------------------------
[array([4000.])]
--------------------------------------
| approxkl           | 8.0366546e-05 |
| clipfrac           | 0.0           |
| explained_variance | -0.0747       |
| fps                | 477           |
| n_updates          | 2             |
| policy_entropy     | 1.385502      |
| policy_loss        | -0.0012774965 |
| serial_timesteps   | 256           |
| time_elapsed       | 0.445         |
| total_timesteps    | 256           |
| value_loss         | 2.5544636     |
--------

[array([4000.])]
--------------------------------------
| approxkl           | 0.00076008687 |
| clipfrac           | 0.0           |
| explained_variance | -0.184        |
| fps                | 443           |
| n_updates          | 17            |
| policy_entropy     | 1.3393639     |
| policy_loss        | -0.0063008573 |
| serial_timesteps   | 2176          |
| time_elapsed       | 4.86          |
| total_timesteps    | 2176          |
| value_loss         | 0.6536979     |
--------------------------------------
[array([5000.])]
--------------------------------------
| approxkl           | 0.0006949943  |
| clipfrac           | 0.0           |
| explained_variance | -0.148        |
| fps                | 416           |
| n_updates          | 18            |
| policy_entropy     | 1.3140092     |
| policy_loss        | -0.0039142864 |
| serial_timesteps   | 2304          |
| time_elapsed       | 5.15          |
| total_timesteps    | 2304          |
| value_loss         | 0.76521

[array([4798.])]
--------------------------------------
| approxkl           | 0.00064091454 |
| clipfrac           | 0.001953125   |
| explained_variance | -0.49         |
| fps                | 443           |
| n_updates          | 33            |
| policy_entropy     | 1.1133368     |
| policy_loss        | -0.0032012297 |
| serial_timesteps   | 4224          |
| time_elapsed       | 9.47          |
| total_timesteps    | 4224          |
| value_loss         | 0.057049584   |
--------------------------------------
[array([7347.])]
--------------------------------------
| approxkl           | 0.0005310583  |
| clipfrac           | 0.0           |
| explained_variance | -7.01         |
| fps                | 453           |
| n_updates          | 34            |
| policy_entropy     | 1.1697947     |
| policy_loss        | -0.0016654879 |
| serial_timesteps   | 4352          |
| time_elapsed       | 9.77          |
| total_timesteps    | 4352          |
| value_loss         | 0.05424

[array([4460.])]
-------------------------------------
| approxkl           | 0.0015808893 |
| clipfrac           | 0.017578125  |
| explained_variance | -1.09        |
| fps                | 498          |
| n_updates          | 49           |
| policy_entropy     | 0.8252347    |
| policy_loss        | -0.011248026 |
| serial_timesteps   | 6272         |
| time_elapsed       | 13.8         |
| total_timesteps    | 6272         |
| value_loss         | 0.0022153296 |
-------------------------------------
[array([5095.])]
-------------------------------------
| approxkl           | 0.0005820569 |
| clipfrac           | 0.0          |
| explained_variance | -0.0634      |
| fps                | 470          |
| n_updates          | 50           |
| policy_entropy     | 0.8980596    |
| policy_loss        | 0.0016251319 |
| serial_timesteps   | 6400         |
| time_elapsed       | 14.1         |
| total_timesteps    | 6400         |
| value_loss         | 0.09808111   |
----------------

[array([5611.])]
--------------------------------------
| approxkl           | 0.00014599603 |
| clipfrac           | 0.0           |
| explained_variance | 0.0606        |
| fps                | 446           |
| n_updates          | 65            |
| policy_entropy     | 1.1871533     |
| policy_loss        | -0.0011475637 |
| serial_timesteps   | 8320          |
| time_elapsed       | 18.3          |
| total_timesteps    | 8320          |
| value_loss         | 1.6236476     |
--------------------------------------
[array([7026.])]
--------------------------------------
| approxkl           | 0.0008130632  |
| clipfrac           | 0.0           |
| explained_variance | -0.0769       |
| fps                | 492           |
| n_updates          | 66            |
| policy_entropy     | 1.174569      |
| policy_loss        | -0.0032713322 |
| serial_timesteps   | 8448          |
| time_elapsed       | 18.6          |
| total_timesteps    | 8448          |
| value_loss         | 0.22690

...

[array([5034.])]
--------------------------------------
| approxkl           | 0.0010148311  |
| clipfrac           | 0.001953125   |
| explained_variance | 0             |
| fps                | 469           |
| n_updates          | 993           |
| policy_entropy     | 0.97933203    |
| policy_loss        | -0.0023423969 |
| serial_timesteps   | 127104        |
| time_elapsed       | 276           |
| total_timesteps    | 127104        |
| value_loss         | 0.057678398   |
--------------------------------------
[array([5818.])]
--------------------------------------
| approxkl           | 0.0019352388  |
| clipfrac           | 0.0390625     |
| explained_variance | 0             |
| fps                | 495           |
| n_updates          | 994           |
| policy_entropy     | 0.99648076    |
| policy_loss        | -0.0076023745 |
| serial_timesteps   | 127232        |
| time_elapsed       | 276           |
| total_timesteps    | 127232        |
| value_loss         | 0.05117

[array([5000.])]
--------------------------------------
| approxkl           | 8.708168e-05  |
| clipfrac           | 0.0           |
| explained_variance | 0.0011        |
| fps                | 484           |
| n_updates          | 1009          |
| policy_entropy     | 1.0784315     |
| policy_loss        | -0.0010705945 |
| serial_timesteps   | 129152        |
| time_elapsed       | 280           |
| total_timesteps    | 129152        |
| value_loss         | 1.8224745     |
--------------------------------------
[array([5000.])]
-------------------------------------
| approxkl           | 0.0010215958 |
| clipfrac           | 0.0          |
| explained_variance | 0.00208      |
| fps                | 471          |
| n_updates          | 1010         |
| policy_entropy     | 1.1229233    |
| policy_loss        | -0.004300418 |
| serial_timesteps   | 129280       |
| time_elapsed       | 281          |
| total_timesteps    | 129280       |
| value_loss         | 2.6934655    |
---

[array([5328.])]
---------------------------------------
| approxkl           | 0.0004027713   |
| clipfrac           | 0.0            |
| explained_variance | 0              |
| fps                | 435            |
| n_updates          | 1025           |
| policy_entropy     | 1.3368064      |
| policy_loss        | -0.00017288479 |
| serial_timesteps   | 131200         |
| time_elapsed       | 285            |
| total_timesteps    | 131200         |
| value_loss         | 0.6035655      |
---------------------------------------
[array([7328.])]
--------------------------------------
| approxkl           | 0.000311972   |
| clipfrac           | 0.0           |
| explained_variance | 0             |
| fps                | 459           |
| n_updates          | 1026          |
| policy_entropy     | 1.3359166     |
| policy_loss        | -0.0010382526 |
| serial_timesteps   | 131328        |
| time_elapsed       | 285           |
| total_timesteps    | 131328        |
| value_loss     

[array([6791.])]
--------------------------------------
| approxkl           | 0.0005841878  |
| clipfrac           | 0.0           |
| explained_variance | 0.0863        |
| fps                | 457           |
| n_updates          | 1041          |
| policy_entropy     | 1.3008953     |
| policy_loss        | -0.0014718252 |
| serial_timesteps   | 133248        |
| time_elapsed       | 289           |
| total_timesteps    | 133248        |
| value_loss         | 1.9202807     |
--------------------------------------
[array([7316.])]
--------------------------------------
| approxkl           | 0.0002193414  |
| clipfrac           | 0.0           |
| explained_variance | -0.251        |
| fps                | 462           |
| n_updates          | 1042          |
| policy_entropy     | 1.2901299     |
| policy_loss        | -0.0013699658 |
| serial_timesteps   | 133376        |
| time_elapsed       | 289           |
| total_timesteps    | 133376        |
| value_loss         | 0.45268

KeyboardInterrupt: 

In [ ]:
#help(model)

def callback(locs, globs):
    hlt = globs['env'].env_method('pl_halite')
    tf.summary.scalar('Player halite', hlt)
    return True

#callback(locals(), globals())

glob = globals()
glob['env'].env_method('pl_halite')

In [ ]:
print(
        "Took {:.2f}s for training - {:.2f} FPS".format(
            total_time_single, n_timesteps / total_time_single
        )
    )

mean_reward = evaluate(model, num_steps=10000)

In [ ]:
obs = env.reset()
for i in range(1000):
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.step(action)
    env.render()


In [ ]:
#obs = env.reset()
action, _states = model.predict(obs)
obs, rewards, dones, info = env.step(action)
print(obs)
#print(action)

In [ ]:
model.save("halite")
del model
model = PPO2.load("halite", env=env)


In [ ]:
env = Single_ship_env_initialization(map_size)

In [ ]:
env.reset()
for i in range(420):
    ob = env.step(env.action_space.sample())
print(ob)

In [ ]:
env.step(env.action_space.sample())
#env.reset()


In [ ]:
x = gym.spaces.Discrete(4)

In [ ]:
x.sample()

In [ ]:
import time
from random import uniform
from IPython.display import display, clear_output

i = 1
for _ in range(200):
    clear_output(wait=True)
    display('Iteration '+str(i)+' Score: '+str(uniform(0, 1)))
    time.sleep(0.1)
    i += 1